In [0]:
# Configuración de conexión JDBC
jdbcHostname = "databricks-sql-server-ac.database.windows.net"  # Servidor SQL
jdbcPort = 1433
jdbcDatabase = "sample_db"  # Nombre exacto de tu base de datos
jdbcUsername = "adminuser"  # Cambiar por tu usuario configurado
jdbcPassword = "Acbp.1995"  # Cambiar por la contraseña configurada

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase}"

# Propiedades de conexión
connectionProperties = {
    "user": jdbcUsername,
    "password": jdbcPassword,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
query = "(SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE') AS t"

df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)

df.show()

+------------+--------------------+
|TABLE_SCHEMA|          TABLE_NAME|
+------------+--------------------+
|     SalesLT|            Customer|
|     SalesLT|        ProductModel|
|     SalesLT|  ProductDescription|
|     SalesLT|             Product|
|     SalesLT|ProductModelProdu...|
|     SalesLT|     ProductCategory|
|         dbo|        BuildVersion|
|         dbo|            ErrorLog|
|     SalesLT|             Address|
|     SalesLT|     CustomerAddress|
|     SalesLT|    SalesOrderDetail|
|     SalesLT|    SalesOrderHeader|
+------------+--------------------+



In [0]:
query = "(SELECT TOP 10 * FROM SalesLT.Product) AS temp"  # Cambia por una tabla válida si es necesario

# Leer datos desde SQL Server
try:
    df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
    df.show()  # Mostrar los datos
except Exception as e:
    print(f"Error al conectar: {e}")

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|        SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|      680|HL Road Frame - B...|   FR-R92B-58|Black|   1059.3100|1431.5000|  58|1016.04|               18|             6|2002-06-01 00:00:00|               null|            null|[47 49 46 38 39 6...|  no_i

In [0]:
query = "(SELECT * FROM SalesLT.Product) AS temp"
df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df.createOrReplaceTempView("Product")


In [0]:
%sql
SELECT ProductID, Name, ListPrice
FROM Product
ORDER BY ListPrice DESC
LIMIT 5;

ProductID,Name,ListPrice
749,"Road-150 Red, 62",3578.2700
753,"Road-150 Red, 56",3578.2700
750,"Road-150 Red, 44",3578.2700
751,"Road-150 Red, 48",3578.2700
752,"Road-150 Red, 52",3578.2700


## 1.Listat todas las tablas disponibles

In [0]:
query = "(SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE') AS temp"
df_tables = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)

# Crear una vista temporal
df_tables.createOrReplaceTempView("temp_tables")

In [0]:
%sql
SELECT TABLE_NAME
FROM temp_tables


TABLE_NAME
Customer
ProductModel
ProductDescription
Product
ProductModelProductDescription
ProductCategory
BuildVersion
ErrorLog
Address
CustomerAddress


## 2. Productos con precios mayores a $50

In [0]:
query = "(SELECT * FROM SalesLT.Product) AS temp"
df_filtered = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df_filtered.createOrReplaceTempView("precios_mayores")

In [0]:
%sql
SELECT ProductID, Name, ListPrice 
FROM precios_mayores
WHERE listPrice > 50;

ProductID,Name,ListPrice
680,"HL Road Frame - Black, 58",1431.5000
706,"HL Road Frame - Red, 58",1431.5000
717,"HL Road Frame - Red, 62",1431.5000
718,"HL Road Frame - Red, 44",1431.5000
719,"HL Road Frame - Red, 48",1431.5000
720,"HL Road Frame - Red, 52",1431.5000
721,"HL Road Frame - Red, 56",1431.5000
722,"LL Road Frame - Black, 58",337.2200
723,"LL Road Frame - Black, 60",337.2200
724,"LL Road Frame - Black, 62",337.2200


## 3. Contar productos por categoría. 

In [0]:
%sql
SELECT ProductCategoryID, COUNT(*)
FROM precios_mayores
GROUP BY ProductCategoryID;

ProductCategoryID,count(1)
31,1
34,1
28,3
27,4
26,7
12,3
22,3
13,2
16,28
6,43


## Contar el total de productos por tamaño.

In [0]:
%sql
SELECT Size, COUNT(*) AS count
FROM precios_mayores
GROUP BY Size
ORDER BY count DESC;

Size,count
null,84
44,29
48,25
52,16
42,15
58,15
38,12
46,11
M,11
L,11


## 6. Calcular el precio promedio de los productos.


In [0]:
%sql
SELECT AVG(ListPrice)
FROM precios_mayores

avg(ListPrice)
744.59522034


In [0]:
%sql
SELECT Name, AVG(ListPrice)
FROM precios_mayores
GROUP BY ProductID, Name;

Name,avg(ListPrice)
"Touring-3000 Blue, 62",742.35000000
ML Mountain Handlebars,61.92000000
"Touring-1000 Blue, 46",2384.07000000
"Touring-2000 Blue, 46",1214.85000000
Hydration Pack - 70 oz.,54.99000000
"Mountain-200 Black, 46",2294.99000000
"HL Touring Frame - Yellow, 60",1003.91000000
"LL Touring Frame - Blue, 50",333.42000000
"Road-650 Red, 58",782.99000000
HL Bottom Bracket,121.49000000


## 7.Encontrar productos sin categoria asignada

In [0]:
%sql
SELECT ProductID, Name, ProductCategoryID
FROM precios_mayores
WHERE ProductCategoryID = "null"

ProductID,Name,ProductCategoryID


## 8. contar productos por color

In [0]:
%sql
SELECT color, count(*) AS count
FROM precios_mayores
GROUP BY color
ORDER BY count DESC;

color,count
Black,89
null,50
Red,38
Silver,36
Yellow,36
Blue,26
Multi,8
Silver/Black,7
White,4
Grey,1


## 9. Calcular el costo total de todos los productos.

In [0]:
%sql
SELECT SUM(StandardCost) as totalCostes
FROM precios_mayores

totalCostes
129275.0025


## 10 Productos que contienen una palabra especifica en su nombre 

In [0]:
%sql
SELECT ProductID, Name, ListPrice
FROM precios_mayores
WHERE Name LIKE '%Helmet%'

ProductID,Name,ListPrice
708,"Sport-100 Helmet, Black",34.9900
711,"Sport-100 Helmet, Blue",34.9900
707,"Sport-100 Helmet, Red",34.9900


## 11. Listar productos de todos los productos.

In [0]:
%sql
SELECT ProductID, Name, SellStartDate
FROM precios_mayores
WHERE SellStartDate >= "2005-01-01"

ProductID,Name,SellStartDate
707,"Sport-100 Helmet, Red",2005-07-01T00:00:00.000+0000
708,"Sport-100 Helmet, Black",2005-07-01T00:00:00.000+0000
709,"Mountain Bike Socks, M",2005-07-01T00:00:00.000+0000
710,"Mountain Bike Socks, L",2005-07-01T00:00:00.000+0000
711,"Sport-100 Helmet, Blue",2005-07-01T00:00:00.000+0000
712,AWC Logo Cap,2005-07-01T00:00:00.000+0000
713,"Long-Sleeve Logo Jersey, S",2005-07-01T00:00:00.000+0000
714,"Long-Sleeve Logo Jersey, M",2005-07-01T00:00:00.000+0000
715,"Long-Sleeve Logo Jersey, L",2005-07-01T00:00:00.000+0000
716,"Long-Sleeve Logo Jersey, XL",2005-07-01T00:00:00.000+0000


## 12. Producto más caro por categoría

In [0]:
%sql
SELECT ProductCategoryID, MAX(ListPrice) AS MaxPrice
FROM precios_mayores
GROUP BY ProductCategoryID
ORDER BY MaxPrice DESC

ProductCategoryID,MaxPrice
6,3578.2700
5,3399.9900
7,2384.0700
18,1431.5000
16,1364.5000
20,1003.9100
12,404.9900
21,357.0600
14,229.4900
31,159.0000


## 13. Calcular el precio promedio por categoría

In [0]:
%sql
SELECT ProductCategoryID, AVG(ListPrice) AS AveragePrice
FROM precios_mayores
GROUP BY ProductCategoryID
ORDER BY AveragePrice DESC

ProductCategoryID,AveragePrice
5,1683.36500000
6,1597.45000000
7,1425.24818182
18,780.04363636
16,678.25357143
20,631.41555556
12,278.99000000
21,220.92928571
14,184.40000000
31,159.00000000


## 14. Encontrar productos descontinuados (Discontinued no es Null)

In [0]:
%sql
SELECT ProductID, Name, DiscontinuedDate
FROM precios_mayores
WHERE DiscontinuedDate !="null"

ProductID,Name,DiscontinuedDate


## 15. Productos con precios mayores que su costo estándar

In [0]:
%sql
SELECT ProductID, Name, ListPrice, StandardCost
FROM precios_mayores
WHERE ListPrice > StandardCost

ProductID,Name,ListPrice,StandardCost
680,"HL Road Frame - Black, 58",1431.5000,1059.3100
706,"HL Road Frame - Red, 58",1431.5000,1059.3100
707,"Sport-100 Helmet, Red",34.9900,13.0863
708,"Sport-100 Helmet, Black",34.9900,13.0863
709,"Mountain Bike Socks, M",9.5000,3.3963
710,"Mountain Bike Socks, L",9.5000,3.3963
711,"Sport-100 Helmet, Blue",34.9900,13.0863
712,AWC Logo Cap,8.9900,6.9223
713,"Long-Sleeve Logo Jersey, S",49.9900,38.4923
714,"Long-Sleeve Logo Jersey, M",49.9900,38.4923
